#  Wprowadzenie do głębokiego uczenia (Deep Learning), perceptronu, MLP




﻿![](https://datascientest.com/en/files/2021/01/Machine-learning-def-.png)



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from typing import Tuple, Any

# Ustawianie losowości dla powtarzalności eksperymentów
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
def load_and_prepare_data() -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """
    Load the Fashion MNIST dataset, split into train/valid/test sets, and normalize the pixel values.

    The data is originally split into a training set of 60,000 images and a test set of 10,000 images.
    We further split the first 5,000 samples from the training set into a validation set, leaving
    55,000 samples for training and 5,000 for validation.

    Returns
    -------
    X_train, X_valid, X_test, y_train, y_valid, y_test : np.ndarray
        Numpy arrays containing the split and normalized data and corresponding labels.
    """
    # Wczytanie zbioru Fashion MNIST
    fashion_mnist = keras.datasets.fashion_mnist
    (X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

    # Podział zbioru treningowego na część walidacyjną (pierwsze 5000) i treningową (pozostałe 55000)
    # Normalizacja pikseli do wartości [0,1]
    X_valid, X_train = X_train_full[:5000]/255.0, X_train_full[5000:]/255.0
    y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
    X_test = X_test / 255.0

    return X_train, X_valid, X_test, y_train, y_valid, y_test

In [ ]:
def show_sample_images(X_train: np.ndarray, y_train: np.ndarray, class_names: list, n_rows: int = 3, n_cols: int = 8):
    """
    Display a grid of sample images with their class labels.

    Parameters
    ----------
    X_train : np.ndarray
        Training images.
    y_train : np.ndarray
        Corresponding training labels.
    class_names : list
        List of class names for each label.
    n_rows : int, optional
        Number of rows in the image grid, by default 3.
    n_cols : int, optional
        Number of columns in the image grid, by default 8.
    """
    plt.figure(figsize=(n_cols * 1.2, n_rows * 1.2))
    for row in range(n_rows):
        for col in range(n_cols):
            index = n_cols * row + col
            plt.subplot(n_rows, n_cols, index + 1)
            plt.imshow(X_train[index], cmap="binary", interpolation="nearest")
            plt.axis('off')
            plt.title(class_names[y_train[index]], fontsize=10)
    plt.subplots_adjust(wspace=0.2, hspace=0.5)
    plt.show()

In [ ]:
def build_model(input_shape: tuple = (28, 28), activation: str = "relu", output_classes: int = 10) -> keras.models.Sequential:
    """
    Build and return a compiled MLP model for Fashion MNIST classification.

    Parameters
    ----------
    input_shape : tuple, optional
        Shape of a single input sample, by default (28, 28)
    activation : str, optional
        Activation function for hidden layers, by default "relu"
    output_classes : int, optional
        Number of output classes, by default 10

    Returns
    -------
    keras.models.Sequential
        A compiled Keras Sequential model.
    """
    model = keras.models.Sequential([
        # Spłaszczenie obrazu 28x28 do wektora 784
        keras.layers.Flatten(input_shape=input_shape),
        # Warstwy ukryte z aktywacją ReLU
        keras.layers.Dense(256, activation=activation),
        keras.layers.Dense(128, activation=activation),
        # Warstwa wyjściowa z softmax do klasyfikacji 10 klas
        keras.layers.Dense(output_classes, activation="softmax")
    ])

    # Kompilacja modelu: używamy cross-entropy do klasyfikacji wieloklasowej, optymalizator SGD
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer="sgd",
                  metrics=["accuracy"])
    return model

In [ ]:
def plot_training_history(history: keras.callbacks.History):
    """
    Plot the training and validation accuracy and loss during model training.

    Parameters
    ----------
    history : keras.callbacks.History
        History object returned by model.fit(), containing training logs.
    """
    pd.DataFrame(history.history).plot(figsize=(8,5))
    plt.grid(True)
    plt.gca().set_ylim(0,1)
    plt.title("Training and Validation Metrics")
    plt.show()

In [ ]:
def predict_and_show(model: keras.models.Sequential, X_new: np.ndarray, y_new: np.ndarray, class_names: list):
    """
    Predict classes for new samples and display the images with their predicted labels.

    Parameters
    ----------
    model : keras.models.Sequential
        The trained Keras model.
    X_new : np.ndarray
        New images to predict.
    y_new : np.ndarray
        True labels for these images.
    class_names : list
        Class names corresponding to the labels.
    """
    y_proba = model.predict(X_new)
    y_pred = np.argmax(y_proba, axis=-1)

    plt.figure(figsize=(len(X_new)*1.5, 1.5))
    for index, image in enumerate(X_new):
        plt.subplot(1, len(X_new), index + 1)
        plt.imshow(image, cmap="binary", interpolation="nearest")
        plt.axis('off')
        plt.title(f"Pred: {class_names[y_pred[index]]}\nTrue: {class_names[y_new[index]]}", fontsize=10)
    plt.subplots_adjust(wspace=0.5)
    plt.show()

In [ ]:

# Klasy nazw dla etykiet Fashion MNIST
class_names = ["T-shirt/Top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [ ]:
# Wczytanie i przygotowanie danych
X_train, X_valid, X_test, y_train, y_valid, y_test = load_and_prepare_data()

In [ ]:
# Wyświetlenie przykładowych obrazów
show_sample_images(X_train, y_train, class_names, n_rows=3, n_cols=8)

In [ ]:
# Budowa i trenowanie modelu MLP
model = build_model(input_shape=[28, 28])
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid))

In [ ]:
# Ewaluacja na zbiorze testowym
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print("Test Accuracy:", test_acc)

In [ ]:
# Wizualizacja przebiegu treningu
plot_training_history(history)

In [ ]:
# Przykładowa predykcja na nowych danych (pierwsze 3 obrazy testowe)
X_new = X_test[:3]
y_new = y_test[:3]
predict_and_show(model, X_new, y_new, class_names)